# Dimension Generation Tutorial

## Learning Objectives

By completing this tutorial, you will:
- ✅ Generate dimension tuples (combinations) using LLMs
- ✅ Convert tuples to natural language queries
- ✅ Validate and filter generated queries
- ✅ Optimize costs with parallel processing
- ✅ Export queries to CSV for systematic testing

## Prerequisites

- Completed [Error Analysis Concepts Tutorial](error_analysis_concepts.md)
- Identified 3-5 key dimensions for query generation
- OpenAI API key configured in `.env` file

## Estimated Time

**Execution Time:** 10-15 minutes (depending on API latency)

**⚠️ Cost Warning:** This notebook makes LLM API calls. Estimated cost: $0.10-0.30 for full execution (using gpt-4o-mini).

---

## Setup and Imports

In [8]:
# Add project root to path for imports
import sys
from pathlib import Path

from langsmith import traceable

sys.path.insert(0, str(Path.cwd().parent.parent))

# Load environment variables
import os

from dotenv import load_dotenv

load_dotenv(Path.cwd().parent.parent / '.env')

# Verify API key is loaded
if not os.environ.get('OPENAI_API_KEY'):
    raise ValueError("OPENAI_API_KEY not found in environment. Check your .env file.")

print("✓ Environment configured")

✓ Environment configured


In [ ]:
# Import required libraries
import json
import time
from typing import Any, Dict, List

# Configure litellm disk cache for cost savings
import litellm
import pandas as pd
from litellm import Cache, completion
from pydantic import BaseModel

litellm.cache = Cache(type="disk")

print("✓ All libraries imported successfully")
print("✓ LiteLLM disk cache configured")

✓ All libraries imported successfully


---

## Why Dimension-Based Query Generation?

In **HW1**, you manually created 10+ diverse queries. This approach:
- ✅ Works for small test sets
- ❌ Doesn't scale to 100+ queries
- ❌ Hard to ensure balanced coverage
- ❌ Time-consuming and error-prone

**Dimension-based generation** solves this:
1. **Define dimensions** (cuisine, dietary, meal type, time, etc.)
2. **Generate tuples** - LLM creates diverse combinations
3. **Generate queries** - LLM converts tuples to natural language
4. **Validate** - Filter unrealistic or duplicate queries
5. **Export** - Save to CSV for systematic testing

**Key Benefits:**
- 🚀 Generate 50-100 queries in minutes
- 📊 Systematic coverage across all dimensions
- 🔄 Reproducible (can regenerate with different seeds)
- ✅ Includes natural language variations

**Reference:** See [`generate_synthetic_queries.py`](generate_synthetic_queries.py) for full implementation.

---

## Step 1: Define Your Dimensions

Based on **error analysis** (HW2 Part 2), identify dimensions where your bot frequently fails or needs comprehensive testing.

For our Recipe Bot, we'll use **6 dimensions**:

In [1]:
# Define dimension values

DIMENSIONS = {
    "DietaryNeedsOrRestrictions": [
        "vegan", "vegetarian", "gluten-free", "dairy-free", "keto", "paleo",
        "halal", "kosher", "no restrictions", "pescatarian", "low-carb",
        "low-sodium", "nut-free", "egg-free", "soy-free", "FODMAP",
        "diabetic-friendly", "high-protein"
    ],
    
    "AvailableIngredientsFocus": [
        "must_use_specific: chicken, rice, vegetables",
        "must_use_specific: pasta, tomatoes, cheese",
        "must_use_specific: eggs, spinach, cheese",
        "must_use_specific: salmon, lemon, herbs",
        "general_pantry: basic ingredients",
        "no_specific_ingredients: open to suggestions"
    ],
    
    "CuisinePreference": [
        "specific_cuisine: Italian",
        "specific_cuisine: Mexican",
        "specific_cuisine: Thai",
        "specific_cuisine: Indian",
        "specific_cuisine: Chinese",
        "specific_cuisine: Mediterranean",
        "specific_cuisine: Japanese",
        "any_cuisine",
        "avoid_specific: spicy"
    ],
    
    "SkillLevelEffort": [
        "beginner_easy_low_effort",
        "intermediate_moderate_effort",
        "advanced_complex_high_effort"
    ],
    
    "TimeAvailability": [
        "quick_under_30_mins",
        "moderate_30_to_60_mins",
        "flexible_no_time_constraint"
    ],
    
    "QueryStyleAndDetail": [
        "short_keywords_minimal_detail",
        "natural_question_moderate_detail",
        "detailed_request_high_detail"
    ]
}

# Display dimension counts
for dim, values in DIMENSIONS.items():
    print(f"{dim}: {len(values)} values")

print(f"\nTotal possible combinations: {18 * 6 * 9 * 3 * 3 * 3:,}")
print("(We'll generate a diverse subset of 10-20 tuples)")

DietaryNeedsOrRestrictions: 18 values
AvailableIngredientsFocus: 6 values
CuisinePreference: 9 values
SkillLevelEffort: 3 values
TimeAvailability: 3 values
QueryStyleAndDetail: 3 values

Total possible combinations: 26,244
(We'll generate a diverse subset of 10-20 tuples)


---

## Step 2: Define Pydantic Models

Pydantic provides **structured output** from LLM responses. This ensures we get valid, parseable data.

In [4]:
# Pydantic models for structured LLM output

class DimensionTuple(BaseModel):
    """Represents one combination of dimension values."""
    DietaryNeedsOrRestrictions: str
    AvailableIngredientsFocus: str
    CuisinePreference: str
    SkillLevelEffort: str
    TimeAvailability: str
    QueryStyleAndDetail: str

class DimensionTuplesList(BaseModel):
    """List of dimension tuples returned by LLM."""
    tuples: List[DimensionTuple]

class QueriesList(BaseModel):
    """List of natural language queries."""
    queries: List[str]

class QueryWithDimensions(BaseModel):
    """Query with associated dimension tuple for tracking."""
    id: str
    query: str
    dimension_tuple: DimensionTuple
    is_realistic_and_kept: int = 1
    notes_for_filtering: str = ""

# Test the models
example_tuple = DimensionTuple(
    DietaryNeedsOrRestrictions="vegan",
    AvailableIngredientsFocus="general_pantry",
    CuisinePreference="specific_cuisine: Italian",
    SkillLevelEffort="beginner_easy_low_effort",
    TimeAvailability="quick_under_30_mins",
    QueryStyleAndDetail="natural_question_moderate_detail"
)

print("Example Dimension Tuple:")
print(example_tuple.model_dump_json(indent=2))

Example Dimension Tuple:
{
  "DietaryNeedsOrRestrictions": "vegan",
  "AvailableIngredientsFocus": "general_pantry",
  "CuisinePreference": "specific_cuisine: Italian",
  "SkillLevelEffort": "beginner_easy_low_effort",
  "TimeAvailability": "quick_under_30_mins",
  "QueryStyleAndDetail": "natural_question_moderate_detail"
}


---

## Step 3: LLM Helper Function

In [13]:
# Configuration
MODEL_NAME = "gpt-4o-mini"  # Cost-effective model for generation

# Note: Cache is already configured globally in Cell 3
# litellm will automatically cache responses based on the messages and model
# Re-running the same prompts will use cached responses (saving API costs)

@traceable(run_type="llm")
def call_llm(messages: List[Dict[str, str]], response_format: Any) -> Any:
    """Call LLM with retry logic for robustness.
    
    Responses are automatically cached on disk. Duplicate requests will
    return cached results, reducing API costs.
    """
    max_retries = 3
    for attempt in range(max_retries):
        try:
            response = completion(
                model=MODEL_NAME,
                messages=messages,
                response_format=response_format
            )
            return response_format(**json.loads(response.choices[0].message.content))
        except Exception as e:
            if attempt == max_retries - 1:
                raise e
            time.sleep(1)  # Wait before retry
    
print("✓ LLM helper function defined (with caching enabled)")

✓ LLM helper function defined (with caching enabled)


---

## Step 4: Generate Dimension Tuples

We'll ask the LLM to generate **diverse combinations** of dimension values.

**Key prompt engineering principles:**
- Emphasize **balanced coverage** (don't over-represent any value)
- Provide **example tuples** showing realistic combinations
- Request **weird/unusual combinations** to test edge cases

In [ ]:
# ========================================
# CONFIGURATION: Demo vs Full Mode
# ========================================

# Set DEMO_MODE = False for full dataset generation
DEMO_MODE = True  # Default: Quick demo for tutorial

if DEMO_MODE:
    NUM_TUPLES = 5  # Generate 5 tuples (15 queries)
    QUERIES_PER_TUPLE = 3
    print("🚀 DEMO MODE: Generating small sample for quick execution")
    print(f"   Tuples: {NUM_TUPLES} | Queries per tuple: {QUERIES_PER_TUPLE}")
    print(f"   Total queries: {NUM_TUPLES * QUERIES_PER_TUPLE}")
    print("   Estimated cost: $0.05-0.10 | Time: ~1-2 minutes")
else:
    NUM_TUPLES = 15  # Generate 15 tuples (45-60 queries)
    QUERIES_PER_TUPLE = 3
    print("📊 FULL MODE: Generating comprehensive query set")
    print(f"   Tuples: {NUM_TUPLES} | Queries per tuple: {QUERIES_PER_TUPLE}")
    print(f"   Total queries: {NUM_TUPLES * QUERIES_PER_TUPLE}")
    print("   Estimated cost: $0.20-0.40 | Time: ~5-10 minutes")

print("\n💡 To switch modes, change DEMO_MODE in this cell and re-run notebook")

In [ ]:
# Generate tuples using configuration
print(f"Generating {NUM_TUPLES} dimension tuples...")
dimension_tuples = generate_dimension_tuples(num_tuples=NUM_TUPLES)

print(f"\n✓ Generated {len(dimension_tuples)} tuples\n")

# Display first 3 tuples
for i, tuple_obj in enumerate(dimension_tuples[:3], 1):
    print(f"Tuple {i}:")
    print(json.dumps(tuple_obj.model_dump(), indent=2))
    print()

In [ ]:
# ========================================
# VALIDATION: Verify tuple generation
# ========================================

# Assert tuples were generated
assert len(dimension_tuples) > 0, "No dimension tuples were generated"
assert len(dimension_tuples) == NUM_TUPLES, f"Expected {NUM_TUPLES} tuples, got {len(dimension_tuples)}"

# Assert all tuples are valid Pydantic models
for i, tuple_obj in enumerate(dimension_tuples):
    assert isinstance(tuple_obj, DimensionTuple), f"Tuple {i} is not a DimensionTuple instance"
    assert len(tuple_obj.DietaryNeedsOrRestrictions) > 0, f"Tuple {i} has empty dietary restriction"
    assert len(tuple_obj.CuisinePreference) > 0, f"Tuple {i} has empty cuisine preference"

print(f"✅ VALIDATION PASSED: Generated {len(dimension_tuples)} valid dimension tuples")

In [15]:
# Generate tuples
print("Generating 10 dimension tuples...")
dimension_tuples = generate_dimension_tuples(num_tuples=10)

print(f"\n✓ Generated {len(dimension_tuples)} tuples\n")

# Display first 3 tuples
for i, tuple_obj in enumerate(dimension_tuples[:3], 1):
    print(f"Tuple {i}:")
    print(json.dumps(tuple_obj.model_dump(), indent=2))
    print()

Generating 10 dimension tuples...

✓ Generated 10 tuples

Tuple 1:
{
  "DietaryNeedsOrRestrictions": "vegan",
  "AvailableIngredientsFocus": "must_use_specific: [tofu, spinach, quinoa]",
  "CuisinePreference": "specific_cuisine: [Indian]",
  "SkillLevelEffort": "beginner_easy_low_effort",
  "TimeAvailability": "quick_under_30_mins",
  "QueryStyleAndDetail": "natural_question_moderate_detail"
}

Tuple 2:
{
  "DietaryNeedsOrRestrictions": "keto",
  "AvailableIngredientsFocus": "general_pantry",
  "CuisinePreference": "any_cuisine",
  "SkillLevelEffort": "intermediate_moderate_effort",
  "TimeAvailability": "flexible_no_time_constraint",
  "QueryStyleAndDetail": "short_keywords_minimal_detail"
}

Tuple 3:
{
  "DietaryNeedsOrRestrictions": "no restrictions",
  "AvailableIngredientsFocus": "no_specific_ingredients",
  "CuisinePreference": "avoid_specific: [Mexican]",
  "SkillLevelEffort": "advanced_complex_high_effort",
  "TimeAvailability": "moderate_30_to_60_mins",
  "QueryStyleAndDetail"

---

## Step 5: Generate Natural Language Queries

Now we convert each dimension tuple into a **realistic natural language query** that a user might ask.

**Key features:**
- Multiple queries per tuple (to increase coverage)
- Natural language variations (typos, lowercase, emojis, text speak)
- Realistic user query styles

In [16]:
def generate_queries_for_tuple(dimension_tuple: DimensionTuple, num_queries: int = 3) -> List[str]:
    """Generate natural language queries for a given dimension tuple."""
    
    prompt = f"""Generate {num_queries} different natural language queries for a recipe chatbot based on these characteristics:
{dimension_tuple.model_dump_json(indent=2)}

The queries should:
1. Sound like real users asking for recipe help
2. Naturally incorporate all the dimension values
3. Vary in style and detail level
4. Be realistic and practical
5. Include natural variations in typing style, such as:
   - Some queries in all lowercase
   - Some with random capitalization
   - Some with common typos
   - Some with missing punctuation
   - Some with emojis or text speak

Generate {num_queries} unique queries that match the given dimensions, varying the text style naturally."""
    
    messages = [{"role": "user", "content": prompt}]
    
    response = call_llm(messages, QueriesList)
    return response.queries

print("✓ Query generation function defined")

✓ Query generation function defined


In [17]:
# Test query generation on first tuple
print("Generating 3 queries for the first dimension tuple...\n")

test_queries = generate_queries_for_tuple(dimension_tuples[0], num_queries=3)

print("Dimension Tuple:")
print(json.dumps(dimension_tuples[0].model_dump(), indent=2))
print("\nGenerated Queries:")
for i, query in enumerate(test_queries, 1):
    print(f"{i}. {query}")

Generating 3 queries for the first dimension tuple...

Dimension Tuple:
{
  "DietaryNeedsOrRestrictions": "vegan",
  "AvailableIngredientsFocus": "must_use_specific: [tofu, spinach, quinoa]",
  "CuisinePreference": "specific_cuisine: [Indian]",
  "SkillLevelEffort": "beginner_easy_low_effort",
  "TimeAvailability": "quick_under_30_mins",
  "QueryStyleAndDetail": "natural_question_moderate_detail"
}

Generated Queries:
1. can u help me find an easy vegan recipe that uses tofu, spinach, and quinoa? I only have like 30 minutes to cook, and I really feel like something Indian tonight!
2. Hey! 🌱 I'm looking for a simple Indian vegan dish. I have some tofu, spinach, and quinoa that I need to use up! Any quick recipes under 30 mins? Thanks!
3. do you have any beginner friendly vegan recipes? i need to use tofu, quinoa and spinach, something easy peasy and quick like less than 30 min. thx!


---

## Step 6: Parallel Processing for Efficiency

Generating queries sequentially is slow. We'll use **ThreadPoolExecutor** to parallelize LLM calls.

**Benefits:**
- 🚀 5-10x faster generation
- 📊 Progress tracking with tqdm
- 🔄 Handles failures gracefully

In [ ]:
# ========================================
# VALIDATION: Verify query generation
# ========================================

# Assert queries were generated
assert len(queries) > 0, "No queries were generated"
expected_count = NUM_TUPLES * QUERIES_PER_TUPLE
assert len(queries) <= expected_count, f"Generated too many queries: {len(queries)} > {expected_count}"
assert len(queries) >= expected_count * 0.8, f"Generated too few queries: {len(queries)} < {expected_count * 0.8} (80% of expected)"

# Assert all queries are valid
for i, query_obj in enumerate(queries):
    assert isinstance(query_obj, QueryWithDimensions), f"Query {i} is not a QueryWithDimensions instance"
    assert len(query_obj.query) >= 10, f"Query {i} is too short: '{query_obj.query}'"
    assert len(query_obj.id) > 0, f"Query {i} has no ID"
    assert query_obj.dimension_tuple is not None, f"Query {i} has no dimension tuple"

# Check for reasonable diversity
unique_queries_count = len(set(q.query for q in queries))
diversity_rate = unique_queries_count / len(queries)
assert diversity_rate >= 0.8, f"Low diversity: only {diversity_rate:.1%} unique queries"

print("✅ VALIDATION PASSED:")
print(f"   - Generated {len(queries)} queries")
print("   - All queries have valid structure")
print(f"   - Diversity rate: {diversity_rate:.1%}")
print(f"   - Average query length: {sum(len(q.query) for q in queries) / len(queries):.0f} chars")

In [ ]:
# Generate all queries in parallel using configuration
queries = generate_queries_parallel(
    dimension_tuples=dimension_tuples,
    queries_per_tuple=QUERIES_PER_TUPLE,
    max_workers=5
)

print(f"\n✓ Generated {len(queries)} total queries from {len(dimension_tuples)} tuples")

In [19]:
# Generate all queries in parallel
queries = generate_queries_parallel(
    dimension_tuples=dimension_tuples,
    queries_per_tuple=3,
    max_workers=5
)

print(f"\n✓ Generated {len(queries)} total queries from {len(dimension_tuples)} tuples")

Generating 3 queries each for 10 dimension tuples...


Generating Queries: 100%|██████████| 10/10 [00:07<00:00,  1.29it/s]


✓ Generated 30 total queries from 10 tuples


---

## Step 7: Quality Validation

Before exporting, check for:
- **Duplicates** - Remove exact duplicate queries
- **Too short** - Filter queries <5 characters
- **Too generic** - Flag vague queries like "recipe"
- **Diversity** - Ensure variety across dimensions

In [20]:
# Check for duplicates
query_texts = [q.query for q in queries]
unique_queries = set(query_texts)

print(f"Total queries: {len(query_texts)}")
print(f"Unique queries: {len(unique_queries)}")
print(f"Duplicates: {len(query_texts) - len(unique_queries)}")

# Check query lengths
query_lengths = [len(q.query) for q in queries]
print("\nQuery length statistics:")
print(f"  Min: {min(query_lengths)} characters")
print(f"  Max: {max(query_lengths)} characters")
print(f"  Average: {sum(query_lengths) / len(query_lengths):.1f} characters")

# Display sample queries
print("\nSample queries:")
for i, q in enumerate(queries[:5], 1):
    print(f"{i}. [{q.id}] {q.query}")

Total queries: 30
Unique queries: 30
Duplicates: 0

Query length statistics:
  Min: 31 characters
  Max: 304 characters
  Average: 132.2 characters

Sample queries:
1. [SYN001] keto recipes using pantry stuff
2. [SYN002] Keto idea? i have some random ingredients!
3. [SYN003] any keto meals that take a bit of effort? 🤔
4. [SYN004] gluten-free recipe pls, flexible ingredients
5. [SYN005] looking for gluten free meals!! got stuff in pantry


---

## Step 8: Export to CSV

Save queries to CSV for use with [`scripts/bulk_test.py`](../../scripts/bulk_test.py).

In [21]:
def save_queries_to_csv(queries: List[QueryWithDimensions], filename: str = "synthetic_queries_for_analysis.csv"):
    """Save generated queries to CSV."""
    
    output_path = Path.cwd() / filename
    
    # Convert to DataFrame
    df = pd.DataFrame([
        {
            'id': q.id,
            'query': q.query,
            'dimension_tuple_json': q.dimension_tuple.model_dump_json(),
            'is_realistic_and_kept': q.is_realistic_and_kept,
            'notes_for_filtering': q.notes_for_filtering
        }
        for q in queries
    ])
    
    # Save to CSV
    df.to_csv(output_path, index=False)
    print(f"✓ Saved {len(queries)} queries to {output_path}")
    
    return df

# Save queries
df = save_queries_to_csv(queries)

✓ Saved 30 queries to /Users/rajnishkhatri/Documents/recipe-chatbot/homeworks/hw2/synthetic_queries_for_analysis.csv


In [22]:
# Display first few rows
print("\nFirst 5 rows of exported CSV:\n")
print(df[['id', 'query']].head())


First 5 rows of exported CSV:

       id                                              query
0  SYN001                    keto recipes using pantry stuff
1  SYN002         Keto idea? i have some random ingredients!
2  SYN003        any keto meals that take a bit of effort? 🤔
3  SYN004       gluten-free recipe pls, flexible ingredients
4  SYN005  looking for gluten free meals!! got stuff in p...


---

## Cost Estimation and Optimization

### Estimated Costs (using gpt-4o-mini)

- **Input tokens:** ~500 tokens per tuple generation + ~300 per query generation
- **Output tokens:** ~200 tokens per tuple generation + ~50 per query
- **Total:** For 10 tuples × 3 queries = ~30,000 tokens
- **Cost:** ~$0.10-0.30 (gpt-4o-mini rates)

### Optimization Tips

1. **Use cheaper models** - gpt-4o-mini instead of gpt-4o
2. **Cache results** - Save generated queries, don't regenerate
3. **Reduce queries_per_tuple** - 1-2 instead of 3-5
4. **Batch processing** - Generate more tuples at once (fewer API calls)
5. **Use local models** - Llama, Mistral (free, but slower/lower quality)

---

## Next Steps

After generating queries:

1. **Run bulk test** on generated queries:
   ```bash
   python scripts/bulk_test.py --queries homeworks/hw2/synthetic_queries_for_analysis.csv
   ```

2. **Perform open coding** on bot responses (see [Error Analysis Concepts](error_analysis_concepts.md))

3. **Build failure taxonomy** (see [Failure Mode Taxonomy Tutorial](failure_mode_taxonomy_tutorial.md))

4. **Iterate:** Generate more queries targeting specific failure modes

---

## Key Takeaways

- ✅ **LLM-based generation scales** - Create 100+ queries in minutes
- ✅ **Dimensions ensure coverage** - Systematic testing across all variations
- ✅ **Parallel processing saves time** - 5-10x faster with ThreadPoolExecutor
- ✅ **Structured output with Pydantic** - Reliable, parseable responses
- ✅ **Natural language variations** - Typos, lowercase, emojis make queries realistic
- ✅ **Cost-effective** - gpt-4o-mini makes this affordable at scale

---

## Further Reading

- [Error Analysis Concepts](error_analysis_concepts.md) - Open and axial coding methodology
- [Failure Mode Taxonomy Tutorial](failure_mode_taxonomy_tutorial.md) - Build structured failure taxonomies
- [generate_synthetic_queries.py](generate_synthetic_queries.py) - Full script implementation
- [HW2 README](README.md) - Complete assignment instructions

---

**Tutorial Status:** ✅ Complete
**Last Updated:** 2025-10-29
**Maintainer:** AI Evaluation Course Team